# Example vega-lite analysis and creation plot for template

example from: https://blog.4dcu.be/programming/2021/05/03/Interactive-Visualizations.html

Turns out source is a data frame!

In [1]:
import pandas as pd
data = pd.read_csv('https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv')

In [6]:
data[:100]

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,NaN,NaN,NaN,N,03/18/2022,NaN,NaN,NaN,NaN,NaN
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,NaN,NaN,NaN,N,08/16/2006,NaN,NaN,NaN,NaN,NaN
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,NaN,NaN,NaN,N,05/26/2006,NaN,NaN,NaN,NaN,NaN
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,NaN,NaN,NaN,N,11/12/2021,NaN,NaN,NaN,NaN,NaN
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,NaN,NaN,NaN,N,05/30/2006,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,293120,COSMO,BCENT SALON/SHOP REGISTRATION,189000991,NOT RENEWED,Y,NaN,BRADLEY D,NaN,HUDSON,...,NaN,NaN,NaN,N,05/30/2006,NaN,NaN,NaN,NaN,NaN
96,1198913,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129437308.0,NOT RENEWED,N,NaN,JOSE L,LASHAY,MORENO,...,NaN,NaN,NaN,N,03/18/2022,NaN,NaN,NaN,NaN,NaN
97,299285,COSMO,LICENSED BARBER,006041203,NOT RENEWED,N,NaN,SARAH,NaN,PRYOR,...,NaN,NaN,NaN,N,05/25/2006,NaN,NaN,NaN,NaN,NaN
98,1125537,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129318887,NOT RENEWED,N,NaN,REBECA,NaN,SERRANO,...,NaN,NaN,NaN,N,07/19/2018,NaN,NaN,NaN,NaN,NaN


In [7]:
data.columns

Index(['_id', 'License Type', 'Description', 'License Number',
       'License Status', 'Business', 'Title', 'First Name', 'Middle',
       'Last Name', 'Prefix', 'Suffix', 'Business Name', 'BusinessDBA',
       'Original Issue Date', 'Effective Date', 'Expiration Date', 'City',
       'State', 'Zip', 'County', 'Specialty/Qualifier',
       'Controlled Substance Schedule',
       'Delegated Controlled Substance Schedule', 'Ever Disciplined',
       'LastModifiedDate', 'Case Number', 'Action', 'Discipline Start Date',
       'Discipline End Date', 'Discipline Reason'],
      dtype='object')

In [10]:
data[0:10]['City']

0              CHICAGO
1                ELGIN
2              CHICAGO
3           SCHAUMBURG
4              CHICAGO
5    LAKE IN THE HILLS
6              CALHOUN
7              CHICAGO
8               GENEVA
9               HERRIN
Name: City, dtype: object

# Geographic Analysis
> How are license holders distributed(City or County)?

In [39]:
city_data = data.groupby('City').size().reset_index(name='counts')
city_data = city_data.sort_values(by='counts', ascending=False)
city_data = city_data[:10]

In [63]:
import altair as alt
input_dropdown = alt.binding_select(options=[], 
                                    name='Statistic')
chart_city = alt.Chart(city_data).mark_bar(size=30).encode( 
  x=alt.X('City', sort='-y', axis=alt.Axis(labelAngle=0)), 
  y=alt.Y('counts:Q',scale=alt.Scale(type='log'),stack=False,
        axis=alt.Axis(
            title='Number of License Holders', 
            grid=True, 
            gridColor='lightgray'
            )),
  tooltip=['City', 'counts']).properties( 
  title='Top 10 Cities by Number of License Holders in Fall 2022, IL',
width=800, height=300)
chart_city


alt.Chart(...)

In [64]:
chart_city.properties(width='container').save("../assets/json/geographic.json")

This bar chart visualizes the "Top 10 Cities by Number of License Holders in Fall 2022, IL," plotting the nominal 'City' category on the x-axis against the quantitative 'counts' on the y-axis. 

Key design choices include using mark_bar() for easy comparison, sorting the x-axis by the y-value ('counts') in descending order, and setting x-axis labels horizontally (labelAngle=0) for readability. A logarithmic scale (type='log') was applied to the y-axis where the top city's count is orders of magnitude larger than the others. A tooltip was also added for interactivity. 

Before visualization, the data was transformed using Python by grouping the original dataset by 'City', aggregating the size of each group into a 'counts' column, sorting this new DataFrame by 'counts' in descending order, and finally, slicing the DataFrame to retain only the top 10 rows.

## Disciplinary Analysis

In [196]:
discipline_data = data.loc[data['Discipline Reason'].notnull()][
    ['License Type', 'Discipline Reason']
].copy()
discipline_data[0:10]


,License Type,Discipline Reason
43,DETECTIVE BOARD,Failed to file and/or pay Illinois income taxes.
48,DETECTIVE BOARD,Failure to file and/or pay Illinois state inco...
111,DETECTIVE BOARD,for being more than 30 days delinquent in the ...
138,COSMO,Obtained license fraudulently by falsifying in...
165,DENTAL,Failure to file and/or pay Illinois state inco...
173,COSMO,after practiced as a nail technician without a...
226,DETECTIVE BOARD,Failing to disclose criminal conviction history.
230,DETECTIVE BOARD,Failed to report his arrest and/or conviction ...
249,DETECTIVE BOARD,due to criminal conviction and failure to report.
267,DETECTIVE BOARD,Criminal conviction history.


In [211]:

select = alt.selection_point(fields=['License Type'], name="select", on="click")
# highlight = alt.selection_point(fields=['License Type'], name="highlight", on="pointerover", empty=False)

stroke_width = (
    alt.when(select).then(alt.value(2, empty=False))
    .otherwise(alt.value(0))
)
base = alt.Chart(discipline_data).add_params(
    select,
    # highlight
)
top_chart = base.mark_bar().encode(
    x=alt.X('License Type:N'),
    y=alt.Y('count():Q', scale=alt.Scale(type='symlog')),
    fillOpacity=alt.when(select).then(alt.value(1)).otherwise(alt.value(0.3)),
    strokeWidth=stroke_width,
    tooltip=['License Type', 'count()']
).add_params(
    select,
    # highlight
).properties(width = 400)
bottom_chart = base.mark_bar().encode(
    x=alt.X('Discipline Reason:N', 
            sort='-y',
            axis=alt.Axis(labelAngle=-45, labelLimit=200, title='Top 5 Discipline Reason')),
    y=alt.Y('count():Q'),
    tooltip=['Discipline Reason:N', 'count():Q']
).transform_filter(
    select
).transform_aggregate(
    count='count()',
    groupby=['Discipline Reason']
).transform_window(
    rank='rank()',
    sort=[alt.SortField('count', order='descending')]
).transform_filter(
    alt.datum.rank <= 5
).properties(width=330)

discipline_chart = top_chart | bottom_chart
discipline_chart

alt.HConcatChart(...)

In [212]:
discipline_chart.properties().save("../assets/json/discipline.json")